In [62]:
benchmarks = pd.read_pickle('julia_benchmarks.pic')
benchmarks

,,Fortran,Julia,Python,R,Matlab,Octave,Mathe-matica,JavaScript,Go,LuaJIT,Java
1,fib,0.70,2.11,77.76,533.52,26.89,9324.35,118.53,3.36,1.86,1.71,1.21
2,parse_int,5.05,1.45,17.02,45.73,802.52,9581.44,15.02,6.06,1.20,5.77,3.35
3,quicksort,1.31,1.15,32.89,264.54,4.92,1866.01,43.23,2.70,1.29,2.03,2.60
4,mandel,0.81,0.79,15.32,53.16,7.58,451.81,5.13,0.66,1.11,0.67,1.35
5,pi_sum,1.00,1.00,21.99,9.56,1.00,299.31,1.69,1.01,1.00,1.00,1.00
6,rand_mat_stat,1.45,1.66,17.93,14.56,14.52,30.93,5.95,2.30,2.96,3.27,3.92
7,rand_mat_mul,3.48,1.02,1.14,1.57,1.12,1.12,1.30,15.07,1.42,1.16,2.36


In [145]:
import time
import random
import numpy as np

In [119]:
from numba import njit, jit

In [120]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


### Utility function

In [121]:
def timer(f, *args, **kwargs):
    start = time.clock()
    ans = f(*args, **kwargs)
    return ans, time.clock() - start

In [130]:
def report(fs, *args, **kwargs):
    ans, t = timer(fs[0], *args, **kwargs)
    print('%s: %.1f' % (fs[0].__name__, 1.0))  
    for f in fs[1:]:
        ans_, t_ = timer(f, *args, **kwargs)
        print('%s: %.1f' % (f.__name__, t/t_))

Fib
----

In [63]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [64]:
fib(20)

6765

In [113]:
%timeit fib(20)

100 loops, best of 3: 5.84 ms per loop


**lru_cache**

In [68]:
from functools import lru_cache

@lru_cache()
def fib_lru(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [114]:
%timeit fib_lru(20)

The slowest run took 92.76 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 213 ns per loop


In [115]:
%%cython -a

cpdef long fib_cython(int n):
    if n<2:
        return n
    return fib_cython(n-1) + fib_cython(n-2)

In [116]:
%timeit fib_cython(20)

10000 loops, best of 3: 129 µs per loop


**numba**

In [82]:
@njit
def fib_numba(n):
    a, b = 0, 1
    for i in range(n-1):
        a, b = b, a + b
    return b

In [83]:
%timeit -r3 -n3 fib_numba(20)

The slowest run took 49263.70 times longer than the fastest. This could mean that an intermediate result is being cached 
3 loops, best of 3: 501 ns per loop


**cython**

In [84]:
%load_ext cython

In [123]:
%%cython -a

def fib_cython_seq(int n):
    cdef int i
    cdef long a, b, tmp
    a = 0
    b = 1
    for i in range(n-1):
        tmp = a
        a = b
        b += tmp
    return b

In [124]:
%timeit fib_cython_seq(20)

The slowest run took 15.42 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 122 ns per loop


In [131]:
report([fib, fib_lru, fib_cython, fib_numba, fib_cython_seq], 20)

fib: 1.0
fib_lru: 1151.4
fib_cython: 26.0
fib_numba: 639.7
fib_cython_seq: 1919.0


Parse_int
----

In [158]:
def parse_int():
    for i in range(1,1000):
        n = random.randint(0,2**31-1)
        s = hex(n)
#         if s[-1]=='L':
#             s = s[0:-1]    
        m = int(s,16)
        assert m == n

In [142]:
%timeit parse_int()

100 loops, best of 3: 4.56 ms per loop


In [159]:
def parse_int_numpy():
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        s = hex(n) 
        m = int(s,16)
        assert m == n

In [160]:
%timeit parse_int_numpy()

1000 loops, best of 3: 1.51 ms per loop


In [169]:
%%cython -a

import cython
import numpy as np
cimport numpy as np

def parse_int_cython():
    cdef int i, n, m
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        m = int(hex(n),16)
        assert m == n

In [170]:
%timeit parse_int_cython()

1000 loops, best of 3: 1.13 ms per loop


In [175]:
report([parse_int, parse_int_numpy, parse_int_cython])

parse_int: 1.0
parse_int_numpy: 2.7
parse_int_cython: 3.2


Quicksort
----

In [171]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

In [190]:
def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

In [191]:
%timeit benchmark_qsort()

10 loops, best of 3: 23.8 ms per loop


In [214]:
%%cython -a

import cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] qsort_kernel_cython(double[:] a, int lo, int hi):
    cdef int i, j
    cdef double pivot

    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_cython():
    lst = np.random.random(5000)
    qsort_kernel_cython(lst, 0, len(lst)-1)

In [215]:
%timeit benchmark_qsort_cython()

The slowest run took 4.02 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 735 µs per loop


In [216]:
report([benchmark_qsort, benchmark_qsort_cython])

benchmark_qsort: 1.0
benchmark_qsort_cython: 25.6


Mandel
----

In [217]:
def mandel(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

In [218]:
def mandelperf():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel(complex(r, i)) for r in r1 for i in r2]

Pi_sum
----

In [ ]:
def pisum(t):
    sum = 0.0
    for j in range(1, 501):
        sum = 0.0
        for k in range(1, t+1):
            sum += 1.0/(k*k)
    return sum

In [ ]:
pisum(10000)

Rand_mat_stat
----

In [ ]:
def randmatstat(t):
    n = 5
    v = np.zeros(t)
    w = np.zeros(t)
    for i in range(1,t):
        a = np.random.randn(n, n)
        b = np.random.randn(n, n)
        c = np.random.randn(n, n)
        d = np.random.randn(n, n)
        P = np.matrix(np.hstack((a, b, c, d)))
        Q = np.matrix(np.vstack((np.hstack((a, b)), np.hstack((c, d)))))
        v[i] = np.trace(np.linalg.matrix_power(np.transpose(P)*P, 4))
        w[i] = np.trace(np.linalg.matrix_power(np.transpose(Q)*Q, 4))
    return (np.std(v)/np.mean(v), np.std(w)/np.mean(w))

In [ ]:
randmatstat(1000)

Rand_mat_mul
----

In [ ]:
def randmatmul(n):
    A = np.random.rand(n,n)
    B = np.random.rand(n,n)
    return np.dot(A,B)

In [ ]:
randmatmul(1000)